**PARTE 1: análisis del corpus**

In [ ]:
import nltk
from nltk import ngrams
from nltk.classify import accuracy, NaiveBayesClassifier
from nltk.corpus import senseval
from nltk.corpus import wordnet

#Descarga de información
nltk.download('senseval')
nltk.download('wordnet')

print('\n\nPalabras del paquete senseval: ', senseval.fileids())


#Se guardan las estructuras correspondientes a cada palabra
interest = senseval.instances('interest.pos')
hard = senseval.instances('hard.pos')
line = senseval.instances('line.pos')
serve = senseval.instances('serve.pos')

#Mostramos un ejemplo del contenido de una instancia de hard.pos con sus diferentes campos
instance_hard = hard[2]
print('\n************************* Información de ejemplo de hard del paquete senseval ****************\n')
print('Información de función de la palabra: ', instance_hard.word)
print('Posición de la palabra en la frase del contexto: ', instance_hard.position)
print('Diferentes sentidos de la palabra ambigua: ', instance_hard.senses)
print('Contexto de la palabra: ', instance_hard.context)
print('****************************************************************************\n')

#devuelve el número de una cadena tipo HARD1
def splitNumero(s):
  if (not '_' in s):
    numero = s.split('.')[2]
  else:
    numero = s.replace('_', '.').split('.')[3]
    #Se busca el índice codificado al final sin ceros y restándole uno para que comience en cero
  return (int(numero) -1)

#devuelve la palabra que antecede al número de una cadena tipo HARD1
def splitPalabra(s):
  #se buscar en wordnet la primera palabra que aparece en el campo de sense
  palabra = s.replace('_', '.').split('.')[0]
  return palabra

#Inicialización variables entidos de las palabras ambiguas: hard, interest, line y serve
num_hard_senses = 0
dict_hard_words = {}
dict_hard_words_num = {}

num_line_senses = 0
dict_line_words = {}
dict_line_words_num = {}

num_interest_senses = 0
dict_interest_words = {}
dict_interest_words_num = {}

num_serve_senses = 0
dict_serve_words = {}
dict_serve_words_num = {}

# A map of SENSEVAL senses to WordNet 3.0 senses.
# SENSEVAL-2 uses WordNet 1.7, which is no longer installable on most modern
# machines and is not the version that the NLTK comes with.
# As a consequence, we have to manually map the following
# senses to their equivalent(s).
SV_SENSE_MAP = {
    "HARD1": ["difficult.a.01"],    # not easy, requiring great physical or mental
    "HARD2": ["hard.a.02",          # dispassionate
              "difficult.a.01"],
    "HARD3": ["hard.a.03"],         # resisting weight or pressure
    "interest_1": ["interest.n.01"], # readiness to give attention
    "interest_2": ["interest.n.03"], # quality of causing attention to be given to
    "interest_3": ["pastime.n.01"],  # activity, etc. that one gives attention to
    "interest_4": ["sake.n.01"],     # advantage, advancement or favor
    "interest_5": ["interest.n.05"], # a share in a company or business
    "interest_6": ["interest.n.04"], # money paid for the use of money
    "cord": ["line.n.18"],          # something (as a cord or rope) that is long and thin and flexible
    "formation": ["line.n.01","line.n.03"], # a formation of people or things one beside another
    "text": ["line.n.05"],                 # text consisting of a row of words written across a page or computer screen
    "phone": ["telephone_line.n.02"],   # a telephone connection
    "product": ["line.n.22"],       # a particular kind of product or merchandise
    "division": ["line.n.29"],      # a conceptual separation or distinction
    "SERVE12": ["serve.v.02"],       # do duty or hold offices; serve in a specific function
    "SERVE10": ["serve.v.06"], # provide (usually but not necessarily food)
    "SERVE2": ["serve.v.01"],       # serve a purpose, role, or function
    "SERVE6": ["service.v.01"]      # be used by; as of a utility
}

def evaluaSentidos(objeto_palabra, nombre_palabra, num_senses, dict_words, dict_words_num):
  for w in objeto_palabra:
    for s in range(len(w.senses)):
      word =  w.senses[s]
      if (word not in dict_words):
        num_sense = splitNumero(SV_SENSE_MAP[word][0])
        short_word = splitPalabra(SV_SENSE_MAP[word][0])
        dict_words.update({word:wordnet.synsets(short_word)[num_sense].definition()})
      if (word not in dict_words_num):
        dict_words_num.update({word:1})
      else:
        dict_words_num.update({word:dict_words_num[word]+1})
  print('\n\n**************** Posibles significados de la palabra', nombre_palabra, ': ******************')
  print(dict_words)
  print('\nNúmero de ocurrencias de los significados de la palabra', nombre_palabra ,':')
  print(dict_words_num)
  print('************************************************************************')
  return (dict_words, dict_words_num)

#LLamadas a la función para las 4 palabras
evaluaSentidos(hard, 'hard', num_hard_senses, dict_hard_words, dict_hard_words_num)
evaluaSentidos(interest, 'interest', num_interest_senses, dict_interest_words, dict_interest_words_num)
evaluaSentidos(serve, 'serve', num_serve_senses, dict_serve_words, dict_serve_words_num)
evaluaSentidos(line, 'line', num_line_senses, dict_line_words, dict_line_words_num)

def calculaVar(dict_var, word):
  for w in word:
    for wc in w.context:
      word_var = wc[0].lower()
      if (word_var in dict_var.keys()):
          dict_var[word_var]+=1
  return dict_var

#Inicilialización del diccionario de cada palabra con sus variaciones y número de ocurrencias
dict_var_hard = {'hard':0, 'harder':0, 'hardest':0}
dict_var_interest = {'interest':0, 'interests':0, 'interested':0}
dict_var_line = {'line':0, 'liner':0,'lines':0, 'lined':0}
dict_var_serve = {'serve':0, 'served':0, 'serving':0, 'serves':0}

calculaVar(dict_var_hard, hard)
calculaVar(dict_var_interest, interest)
calculaVar(dict_var_line, line)
calculaVar(dict_var_serve, serve)

print('\n\n***************************************************************************************')
print('\tOCURRENCIAS DE VARIACIONES GRAMATICALES DE LAS PALABRAS')
print('\nOcurrencias de las variaciones de hard: ', dict_var_hard)
print('Ocurrencias de las variaciones de serve: ', dict_var_serve)
print('Ocurrencias de las variaciones de line: ', dict_var_line)
print('Ocurrencias de as variaciones de interest: ', dict_var_interest)
print('\n***************************************************************************************\n')


#Datos erróneos, en la instancia 999 se comprueba que en vez de una tupla (palabra, categoría gramatical) se tiene ta solola palabra clave FRASL
print('\n\n***********************************************\nDatos erróneos:')
def checkFormat(word, word_text):
  i=0
  fine_word = []
  for w in word:
    for wc in w.context:
      if (len(wc) !=2):
        print('Instancia nº ', i, ' de  ',word_text, ' mal formateada.')
        break
      fine_word.append(w)
    i+=1
  return fine_word

print('\n\nInstancias de hard antes de checkear las etiquetas  del contexto: ', len(hard))
hard = checkFormat(hard, 'hard')
print('\nInstancias de hard después de checkear las etiquetas  del contexto: ', len(hard))

print('\nInstancias de line antes de checkear las etiquetas  del contexto: ', len(line))
line = checkFormat(line, 'line')
print('\nInstancias de line después de checkear las etiquetas  del contexto: ', len(line))

print('\nInstancias de serve antes de checkear las etiquetas  del contexto: ', len(serve))
serve = checkFormat(serve, 'serve')
print('\nInstancias de serve después de checkear las etiquetas  del contexto: ', len(serve))

print('\nInstancias de interest antes de checkear las etiquetas  del contexto: ', len(interest))
interest = checkFormat(interest, 'interest')
print('\nInstancias de interest después de checkear las etiquetas  del contexto: ', len(interest))

#hard.remove(999)


[nltk_data] Downloading package senseval to /root/nltk_data...
[nltk_data]   Package senseval is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Palabras del paquete senseval:  ['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']

************************* Información de ejemplo de hard del paquete senseval ****************

Información de función de la palabra:  hard-a
Posición de la palabra en la frase del contexto:  3
Diferentes sentidos de la palabra ambigua:  ('HARD1',)
Contexto de la palabra:  [('i', 'PRP'), ('find', 'VBP'), ('it', 'PRP'), ('hard', 'JJ'), ('to', 'TO'), ('believe', 'VB'), ('that', 'IN'), ('the', 'DT'), ('sacramento', 'NNP'), ('river', 'NNP'), ('will', 'MD'), ('ever', 'RB'), ('be', 'VB'), ('quite', 'RB'), ('the', 'DT'), ('same', 'JJ'), (',', ','), ('although', 'IN'), ('i', 'PRP'), ('certainly', 'RB'), ('wish', 'VBP'), ('that', 'IN'), ('i', 'PRP'), ("'m", 'VBP'), ('wrong', 'JJ'

**Parte 2: extracción de características**

In [ ]:
import string

hard_common = nltk.FreqDist([w.context[0] for w in hard])
line_common = nltk.FreqDist([w.context[0] for w in line])
interest_common = nltk.FreqDist([w.context[0] for w in interest])
serve_common = nltk.FreqDist([w.context[0] for w in serve])

hard_common = hard_common.most_common()
line_common = line_common.most_common()
interest_common = interest_common.most_common()
serve_common = serve_common.most_common()

print('Hard common: ', hard_common)
print('Line common: ', line_common)
print('Interest common: ', interest_common)
print('serve common: ', serve_common)

hard_most_common = []
line_most_common = []
interest_most_common = []
serve_most_common = []

STOP_FORMS = ['PRP$','RB','EX','WP','CD','TO','WRB','PRP','IN','DT','CC']
STOP_WORDS = ['q','most','that','which','today', "'d", "'ll", "'m", "'re", "'s", "'t", "'ve", '000', '1', '10', '2', 'I', 'also', "don'", 'n', 'one', 'said', 'say', 'says', 'us']
STOP_PUNCTUATION = ['``', '--', "''"]
#Función que elimina de la lista de palabras más comunes preposiciones, determinantes, etc.
def skipStopWords(common, amb_words):
  most_common = []
  for w in common:
    if ( (w[0][1] not in STOP_FORMS) and (w[0][0] not in STOP_WORDS) and (w[0][0] not in string.punctuation) and (w[0][0] not in STOP_PUNCTUATION) and (w[0][0] not in amb_words) ):
      most_common.append(w[0][0])
  return most_common

hard_most_common = skipStopWords(hard_common, dict_var_hard.keys())
line_most_common = skipStopWords(line_common, dict_var_line.keys())
serve_most_common = skipStopWords(serve_common, dict_var_serve.keys())
interest_most_common = skipStopWords(interest_common, dict_var_interest.keys())

print('\nPalabras más comunes en hard: ', hard_most_common)
print('\nPalabras más comunes en line: ', line_most_common)
print('\nPalabras más comunes en serve: ', serve_most_common)
print('\nPalabras más comunes en interest: ', interest_most_common)

#Se establece el número de palabras más frecuentes a 250
N=250
hard_250_common = hard_most_common[0:N]
serve_250_common = serve_most_common[0:N]
interest_250_common = interest_most_common[0:N]
line_250_common = line_most_common[0:N]
print('\nLista 6 palabras más comunes ***************************\n')
print('Lista de 6 palabras más comunes en hard: ', hard_250_common)
print('Lista de 6 palabras más comunes en line: ', line_250_common)
print('Lista de 6 palabras más comunes en serve: ', serve_250_common)
print('Lista de 6 palabras más comunes en interest: ', interest_250_common)
print('************************************************\n\n')

window_size = 2
#Lista de diccionarios de ngrams para las palabras ambiguas
ngram_pos_hard = []
ngram_pos_line = []
ngram_pos_serve = []
ngram_pos_interest = []

#Función que crea un diccionario con la aparición de las dos palabras posteriores y anteriores a la palabra ambigua

def prevNextWords(amb_word):
  i=0
  #Array con los diccionarios de cada palabra ambigua
  ngram_pos = []
  for w in amb_word:
    #diccionario de colocación con ventana 2 para cada contexto
    ngram = {}
    #posición  de la palabra ambigua en el contexto
    pos = int(w.position)
    context_size = len(w.context)-1
    if (pos-window_size >= 0):
      ngram.update({'contains('+w.context[pos-1][0]+')': True})
      ngram.update({'contains('+w.context[pos-2][0]+')': True})
    elif (pos-1 >=0):
      ngram.update({'contains('+w.context[pos-1][0]+')': True})
    if (pos+window_size <= context_size):
      ngram.update({'contains('+w.context[pos+1][0]+')': True})
      ngram.update({'contains('+w.context[pos+2][0]+')': True})
    elif (pos+1 <=context_size):
      ngram.update({'contains('+w.context[pos+1][0]+')': True})
    ngram_pos.append(ngram)
  return ngram_pos

ngram_pos_hard = prevNextWords(hard)
ngram_pos_line = prevNextWords(line)
ngram_pos_serve = prevNextWords(serve)
ngram_pos_interest = prevNextWords(interest)

print('\n\nEjemplos de diccionarios de colocación de las palabras ambiguas:')
print('Hard: ', ngram_pos_hard[0])
print('Line: ', ngram_pos_line[0])
print('Serve: ', ngram_pos_serve[0])
print('Interest: ', ngram_pos_interest[0])


Hard common:  [(('``', '``'), 22291), (('it', 'PRP'), 10904), (('the', 'DT'), 9655), (('but', 'CC'), 5036), (('i', 'PRP'), 2744), (('in', 'IN'), 2531), (('and', 'CC'), 1943), (('(', '('), 1753), (('if', 'IN'), 1493), (('he', 'PRP'), 1476), (('a', 'DT'), 1257), (('as', 'IN'), 1109), (('they', 'PRP'), 1080), (('that', 'DT'), 1058), (('there', 'EX'), 1030), (('for', 'IN'), 996), (('when', 'WRB'), 947), (('this', 'DT'), 930), (('she', 'PRP'), 927), (('even', 'RB'), 885), (('although', 'IN'), 841), (('with', 'IN'), 837), (('we', 'PRP'), 784), (('after', 'IN'), 740), (('while', 'IN'), 699), (('now', 'RB'), 556), (('one', 'CD'), 542), (('so', 'RB'), 540), (('to', 'TO'), 508), (('some', 'DT'), 497), (('what', 'WP'), 477), (('on', 'IN'), 468), (('since', 'IN'), 453), (('because', 'IN'), 438), (('still', 'RB'), 435), (('yet', 'RB'), 427), (('not', 'RB'), 402), (('you', 'PRP'), 372), (('these', 'DT'), 347), (('from', 'IN'), 312), (('most', 'JJS'), 290), (('at', 'IN'), 277), (('no', 'DT'), 270), (

**Parte 3: entrenamiiento de clasificadores**

In [ ]:
import random

#vectoor de características de colocación inicializados para cada palabra ambigua
features_col_hard = []
features_col_serve = []
features_col_line = []
features_col_interest = []

#Funciión que crea para cada sentido ambiguo de una palabra le asigna un vector de características de colocación
def createColData(word, features_w):
  i_w=0
  features_col = []
  for w in word:
    #Capturo solamente el primer sentido en caso de que haya varios
    sense = w.senses[0]
    #dict con las palabras previas y posteriiores
    features = features_w[i_w]
    features_col.append([features, sense])
    i_w+=1
  return features_col

features_col_hard = createColData(hard, ngram_pos_hard)
features_col_serve = createColData(serve, ngram_pos_serve)
features_col_line = createColData(line, ngram_pos_line)
features_col_interest = createColData(interest, ngram_pos_interest)

#se mezclan las instancias
random.shuffle(features_col_hard)
random.shuffle(features_col_line)
random.shuffle(features_col_interest)
random.shuffle(features_col_serve)

features_col_hard_train = []
features_col_hard_test = []

features_col_line_train = []
features_col_line_test = []

features_col_serve_train = []
features_col_serve_test = []

features_col_interest_train = []
features_col_interest_test = []

#20% para test, 80% para validación
features_col_hard_train = features_col_hard[:int(0.8*len(features_col_hard))]
features_col_hard_test = features_col_hard[int(0.8*len(features_col_hard)):]

features_col_line_train = features_col_line[:int(0.8*len(features_col_line))]
features_col_line_test = features_col_line[int(0.8*len(features_col_line)):]

features_col_serve_train = features_col_serve[:int(0.8*len(features_col_serve))]
features_col_serve_test = features_col_serve[int(0.8*len(features_col_serve)):]

features_col_interest_train = features_col_interest[:int(0.8*len(features_col_interest))]
features_col_interest_test = features_col_interest[int(0.8*len(features_col_interest)):]

nbc_col_hard = NaiveBayesClassifier.train(features_col_hard_train)
acc_nbc_col_hard = accuracy(nbc_col_hard, features_col_hard_test)

nbc_col_line = NaiveBayesClassifier.train(features_col_line_train)
acc_nbc_col_line = accuracy(nbc_col_line, features_col_line_test)

nbc_col_serve = NaiveBayesClassifier.train(features_col_serve_train)
acc_nbc_col_serve = accuracy(nbc_col_serve, features_col_serve_test)

nbc_col_interest = NaiveBayesClassifier.train(features_col_interest_train)
acc_nbc_col_interest = accuracy(nbc_col_interest, features_col_interest_test)

print('La accuracy para característcas de colocación de hard es: ', acc_nbc_col_hard)
print('La accuracy para característcas de colocación de line es: ', acc_nbc_col_line)
print('La accuracy para característcas de colocación de serve es: ', acc_nbc_col_serve)
print('La accuracy para característcas de colocación de interest es: ', acc_nbc_col_interest)

#Coprueba para cada instancia de la palabra ambigua si aparecen las 250 palabras más ambiiguas en el contexto
def fillFeaturesCommon(word, common_250):
  dict_array_common = []
  for w in word:
    dict_common = {}
    #Inicializo a falso las apariciones de las 6 palabras más comunes en el contexto
    for ws in common_250:
      dict_common.update({ws: False})
    for wc in w.context:
      #Si la palabra del contexto se corresponde con unas de las ás communes se indica su aparición en el dict
      if (wc[0] in dict_common.keys()):
        dict_common.update({wc[0]: True})
    #Agrego el dict al array de las instancias de la palabra ambigua
    dict_array_common.append(dict_common)
  return dict_array_common

dict_hard_array_common = []
dict_line_array_common = []
dict_serve_array_common = []
dict_interest_array_common = []

dict_hard_array_common = fillFeaturesCommon(hard, hard_250_common)
dict_line_array_common = fillFeaturesCommon(line, line_250_common)
dict_serve_array_common = fillFeaturesCommon(serve, serve_250_common)
dict_interest_array_common = fillFeaturesCommon(interest, interest_250_common)

print('\n\nInstancias de ejemplo de entrenamiento para hard:',len(dict_hard_array_common))
features_common_hard = []
features_common_line = []
features_common_serve = []
features_common_interest = []

def createCommonData(word, dict_array_common):
  i_w=0
  features_col = []
  for w in word:
    #Capturo solamente el primer sentido en caso de que haya varios
    sense = w.senses[0]
    #dict con las palabras previas y posteriiores
    features_col = dict_array_common[i_w]
    features_common_hard.append([features_col, sense])
    i_w+=1
  return features_common_hard

features_common_hard = createCommonData(hard, dict_hard_array_common)
features_common_line = createCommonData(line, dict_line_array_common)
features_common_serve = createCommonData(hard, dict_serve_array_common)
features_common_interest = createCommonData(interest, dict_interest_array_common)

#se mezclan las instancias
random.shuffle(features_common_hard)
random.shuffle(features_common_line)
random.shuffle(features_common_serve)
random.shuffle(features_common_interest)

#20% para test, 80% para validación
features_common_hard_train = features_common_hard[:int(0.8*len(features_common_hard))]
features_common_hard_test = features_common_hard[int(0.8*len(features_common_hard)):]

features_common_line_train = features_common_line[:int(0.8*len(features_common_line))]
features_common_line_test = features_common_line[int(0.8*len(features_common_line)):]

features_common_serve_train = features_common_serve[:int(0.8*len(features_common_serve))]
features_common_serve_test = features_common_serve[int(0.8*len(features_common_serve)):]

features_common_interest_train = features_common_interest[:int(0.8*len(features_common_interest))]
features_common_interest_test = features_common_interest[int(0.8*len(features_common_interest)):]


nbc_common_hard = NaiveBayesClassifier.train(features_common_hard_train)
acc_nbc_common_hard = accuracy(nbc_common_hard, features_common_hard_test)

nbc_common_line = NaiveBayesClassifier.train(features_common_line_train)
acc_nbc_common_line = accuracy(nbc_common_line, features_common_line_test)

nbc_common_serve = NaiveBayesClassifier.train(features_common_serve_train)
acc_nbc_common_serve = accuracy(nbc_common_serve, features_common_serve_test)

nbc_common_interest = NaiveBayesClassifier.train(features_common_interest_train)
acc_nbc_common_interest = accuracy(nbc_common_interest, features_common_interest_test)

print('\n\nLa accuracy para bag of words de hard es: ', acc_nbc_common_hard)
print('La accuracy para bag of words de line es: ', acc_nbc_common_line)
print('La accuracy para bag of words de serve es: ', acc_nbc_common_serve)
print('La accuracy para bag of words de interest es: ', acc_nbc_common_interest)

La accuracy para característcas de colocación de hard es:  0.9682883679787705
La accuracy para característcas de colocación de line es:  0.9470150480080757
La accuracy para característcas de colocación de serve es:  0.9551982200647249
La accuracy para característcas de colocación de interest es:  0.9766643788606726


Instancias de ejemplo de entrenaiento para hard: 113050


KeyboardInterrupt: ignored

**Parte 3: respuesta a cuestiones**

*¿Cuál es el conjunto de características que aporta mejores resultados? ¿Por qué? *

- El conjunto de características que apoorta mejores resultados es el de har de colocación, ya que solo tiene 3 posibles sentidos y además éstoos  no son múltiiples.

*¿Cuál es el sentido más difícil de identificar? ¿Por qué?*

- El sentido mmás difícil de identificar es el de line, ya que tiene mmuchos sentidos y algunos son múltiples.

*¿Qué posibles mejoras se podrían aplicar para mejorar el rendimiento de los clasificadores creados? No es necesario que las implementes, solo que las comentes.*

- Para las características de colocación se podrían excluir palabras como determinantes, artículos, caracteres de puntuación, etc que estuviesen colocados  antes o después de la palabra ambigua  o bien aumentar la  ventana (windows_size). Paralas característiicas basadas en bags of words podría aumentarse el número de palabras del vocabulario o ponderar estas de algún modo por el número de apariciones de las palabras en el contexto o su distancia a la palabra ambigua.


*¿Por qué no es justo comparar directamente la exactitud aportada por los clasificadores que han aprendido diferentes palabras ambiguas?*

- Porque la colocación y las palabras más comunes dependen de las frases de contexto, y estas son arbistrarias, de ejemplo. Además hay sentidos múltiples que dificultan aún más la tarea.

*¿Cómo podrías hacerlo para que la comparación entre clasificadores que desambiguan palabras diferentes tenga sentido?*

- Al existir en un sentido significados múltiples podría elegirse el sentido más común de entre los posibles o proporcioonar frases de coontextoopara cada uno de los posibles sentidos de la palabra ambigua.

*Compara la exactitud de los clasificadores con la que proporcionaría un clasificador que asignara el sentido de forma aleatoria. ¿Cuál sería el mejor clasificador tomando como referencia (baseline), el clasificador aleatorio?*

- Un clasificador que asignase el sentido de forma aleatoria llevado al límite sería peor, puesto que no habría características comunes ni uniiformidad para formar los vectores de características (colocación y bag of words). El clasificador aleatorio funcionaría mejor parapalabras con pocos sentidos y sin sentidos múltiples.


**Parte 4:**

*¿Cuáles son las limitaciones de los clasificadores que has creado para la desambiguación del sentido de las palabras?*

- No aprende sentidos múltiples, sino  que se toma como referencia si hay varios sentidos el primero de ellos. Por motivos de tiempo de procesamiento podría aumentarse el tamaño del window_size y aumentar la accuracy del clasificador. 